# Initialization

In [ ]:
import torch
import torch.nn as nn
# from torch.optim import Adam

# Self-attention module

In [ ]:
class SelfAttention(nn.Module):

    def __init__(self, embed_size, heads):

        super(SelfAttention, self).__init__()

        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (self.head_dim * heads == embed_size), "Embed size needs to be divisible by heads"

        # Working individually in each head
        self.value_weights  = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.key_weights    = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.query_weights = nn.Linear(self.head_dim, self.head_dim, bias=False)

        # Concatenating the heads with Wz
        self.fc_out  = nn.Linear(heads*self.head_dim, self.embed_size)

    def forward(self, values, keys, query, mask):

        N = query.shape[0] # 2
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1] # 9

        # Current shape of keys, query and values = (2, 9, 512) [Batch, Sequence_length, Embed_size]
        
        # Split embedding into multiple heads

        values  = values.reshape(N, value_len, self.heads, self.head_dim) # Shape = (2,9,4,128)
        keys    = keys.reshape(N, key_len, self.heads, self.head_dim) # Shape = (2,9,4,128)
        queries = query.reshape(N, query_len, self.heads, self.head_dim) # Shape = (2,9,4,128)

        values = self.value_weights(values)
        keys = self.key_weights(keys)
        queries = self.query_weights(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk",[queries, keys]) # Shape = [2, 4, 9, 9]
        # queries shape: (N, query_len, heads, head_dim)
        # keys shape:    (N, key_len,   heads, head_dim)
        # energy shape:  (N, heads, query_len, key_len)
       
        if mask is not None:  # Masking for decoder block and BERT
            energy = energy.masked_fill(mask==0, float("-1e20"))

        attended = torch.softmax(energy / (self.embed_size**0.5), dim=3) # Shape = [2, 4, 9, 9]  Why we are taking the dim as 3? https://stackoverflow.com/questions/49036993/pytorch-softmax-what-dimension-to-use
        # We are keeping the softmax dim as 3, i.e. keys, because we want the total prob of keys' association with the query as 1
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd",[attended, values]).reshape(N, query_len, self.heads*self.head_dim)  # Shape = (2, 9, 512)
        # attention shape: (N, heads, query_len, key_len)
        # values shape :   (N, value_len,   heads, head_dim)
        # out shape: (N, query_len, heads, head_dim) then flatten by concatenating

        out = self.fc_out(out)  # Shape = (2, 9, 512)

        return out

# Internal transformer block

In [ ]:
class TransformerBlock(nn.Module):

    def __init__(self, embed_size, heads, dropout, forward_expansion):

        super(TransformerBlock, self).__init__()

        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion*embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion*embed_size, embed_size)
        )

        self.drop = nn.Dropout(dropout)

        self.norm2 = nn.LayerNorm(embed_size) ###### 512!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!    Each word is normalised in the sequence on its own

    def forward(self, value, key, query, mask):
    
    # Receiving the word + positional embeddings combined , mask

        attention = self.attention(value, key, query, mask) # Shape: (2,9,512) ###################### Attention layer

        x = self.drop(self.norm1(attention + query))        # Normalization layer
        
        forward = self.feed_forward(x)                      # MLP       

        out = self.drop(self.norm2(forward + x)) # Shape: (2,9,512) #Normalization layer

        # print("Attention: ", attention)
        # print("Query: ", query)
        # print("Key: ", key)
        # print("Value: ", value)

        return out

# Encoder

In [ ]:
class Encoder(nn.Module):

    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length):

        super(Encoder, self).__init__()

        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size) # 10 , 512
        self.position_embedding = nn.Embedding(max_length, embed_size) # 100, 512

        self.layers = nn.ModuleList(
            [
             TransformerBlock(
                 embed_size,
                 heads,
                 dropout=dropout,
                 forward_expansion = forward_expansion
             ) for _ in range(num_layers)
            ]
        )

        self.drop = nn.Dropout(dropout)

    def forward(self, x, mask):

        N, seq_length = x.shape  # 2, 9

        positions = torch.arange(0,seq_length).expand(N, seq_length).to(device)       # Tensor:    [[0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]]
        
        out = self.drop(self.word_embedding(x) + self.position_embedding(positions))  # out is sum of 'word' & 'position' embedding values with shape [2,9,512]

        for layer in self.layers: 
            out = layer(out, out, out, mask) # value, key, query, mask

        return out

# Encoder init

In [ ]:
device = torch.device("cuda")
enc = Encoder(
        src_vocab_size=10,   # This is the max
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=4,
        dropout=0,
        device=torch.device("cuda"),
        max_length=100)
enc = enc.to(device)
x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)

In [ ]:
encoded_input = enc(x, mask=None)

In [ ]:
encoded_input.shape

torch.Size([2, 9, 512])

# Decoder

In [ ]:
class DecoderBlock(nn.Module):

    def __init__(self, embed_size, heads, forward_expansion, dropout, device):

        super(DecoderBlock, self).__init__()

        self.attention = SelfAttention(embed_size, heads)
        self.norm = nn.LayerNorm(embed_size)
        # self.dropout = dropout
        self.transformer_block =TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.drop = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):

        # Masked multi-head attention
        attention = self.attention(x,x,x,trg_mask) # value, key, query, mask

        query = self.drop(self.norm(attention+x))

        # Main transformer block including the interaction attention block
        # Note that the query is coming from the MM attention unit, i.e. indirectly the target
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [ ]:
class Decoder(nn.Module):

    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length
        ):

        super(Decoder, self).__init__()

        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [DecoderBlock(
                embed_size, heads, forward_expansion, dropout, device
                ) for _ in range(num_layers)
            ]
        )

        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.drop = nn.Dropout(dropout)
    
    def forward(self, x, enc_out, src_mask, trg_mask):
        
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device) # absolutely similar to encoder
        x = self.drop((self.word_embedding(x) + self.position_embedding(positions)))

        # Entering into the decoder block
        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask) # x, value, key
            # Here x is from the trg dataset. value and key are encoder outputs coming from the encoder block
            # x goes into the MM attention unit and generates the query, value and key are used from the encoded data to predict the next target
        
        out = torch.softmax(self.fc_out(x), dim=2)

        return out

# Transformer

In [ ]:
class Transformer(nn.Module):
    
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=4,
        dropout=0,
        device="cuda",
        max_length=100):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length)
        
        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length)
        
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src!= self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones(trg_len, trg_len)).expand(N,1,trg_len, trg_len)

        return trg_mask.to(device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src  = self.encoder(src, src_mask)
        out      = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

# Transformer init

In [ ]:
device = torch.device("cuda")
x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
trg = torch.tensor([[0, 1, 7, 4, 3, 5, 9, 2, 0], [0, 1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

src_pad_idx, trg_pad_idx = 0, 0
src_vocab_size, trg_vocab_size = 10, 10

model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx).to(device)

out = model(x, trg[:,:-1]).argmax(axis=2)
print(out.shape)

torch.Size([2, 8])


In [ ]:
out

tensor([[5, 5, 2, 2, 9, 4, 2, 2],
        [5, 6, 8, 8, 8, 2, 3, 7]], device='cuda:0')